In [181]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

import numpy as np
import pandas as pd
import os
import time
import re

# 1 读入数据 处理数据

## 1.1  文本处理

In [182]:
df = pd.read_csv("poem1.txt",sep='\t',encoding="utf-8",header=None)
df

,0,1,2
0,张九龄,感遇·其一,"兰叶春葳蕤,桂华秋皎洁.欣欣此生意,自尔为佳节.谁知林栖者,闻风坐相悦."
1,张九龄,感遇·其二,"江南有丹桔,经冬犹绿林.岂伊地气暖,自有岁寒心.可以荐佳客,奈何阻重深.运命唯所遇,循环不可..."
2,李白,下终南山过斛斯山人宿置酒,"暮从碧山下,山月随人归.却顾所来径,苍苍横翠微.相携及田家,童稚开荆扉.绿竹入幽径,青萝拂行..."
3,李白,月下独酌,"花间一壶酒,独酌无相亲.举杯邀明月,对影成三人.月既不解饮,影徒随我身.暂伴月将影,行乐须及..."
4,李白,春思,"燕草如碧丝,秦桑低绿枝.当君怀归日,是妾断肠时."
5,杜甫,望岳,"岱宗夫如何,齐鲁青未了.造化钟神秀,阴阳割昏晓.荡胸生层云,决眦入归鸟.会当凌绝顶,一览众山小."
6,杜甫,赠卫八处士,"人生不相见,动如参与商.今夕复何夕,共此灯烛光.少壮能几时,鬓发各已苍.访旧半为鬼,惊呼热中..."
7,杜甫,佳人,"绝代有佳人,幽居在空谷.自云良家女,零落依草木.关中昔丧乱,兄弟遭杀戮.官高何足论,不得收骨..."
8,杜甫,梦李白·其一,"死别已吞声,生别常恻恻.江南瘴疠地,逐客无消息.故人入我梦,明我常相忆.恐非平生魂,路远不可..."
9,杜甫,梦李白·其二,"浮云终日行,游子久不至.三夜频梦君,情亲见君意.告归常局促,苦道来不易.江湖多风波,舟楫恐失..."


In [183]:
a="兰叶春葳蕤,桂华秋皎洁.欣欣此生意,自尔为佳节.谁知林栖者,闻风坐相悦."
re.split("\.",a)

['兰叶春葳蕤,桂华秋皎洁', '欣欣此生意,自尔为佳节', '谁知林栖者,闻风坐相悦', '']

In [184]:
def is_Chinese(word):
    for ch in word:
        if '\u4e00' <= ch <= '\u9fff':
            return True
    return False

is_Chinese("《满江红》")

True

In [185]:
poem_seq_list=[]
poem_string=""
poem_char_dic={}
for i,r in df.iterrows():
#     print(i)
    poem_seq = r[2]
    poem_seg=re.split(r"\.",poem_seq)
    for seq in poem_seg:
        seq=seq.replace(",","")
        if len(seq)==10:  # 5言律诗
            poem_seq_list.append([i,seq])
            poem_string+=seq
            for c in seq:
                if c in poem_char_dic:
                    poem_char_dic[c]+=1
                else:
                    poem_char_dic[c]=1
poem_seq_list

[[0, '兰叶春葳蕤桂华秋皎洁'],
 [0, '欣欣此生意自尔为佳节'],
 [0, '谁知林栖者闻风坐相悦'],
 [1, '江南有丹桔经冬犹绿林'],
 [1, '岂伊地气暖自有岁寒心'],
 [1, '可以荐佳客奈何阻重深'],
 [1, '运命唯所遇循环不可寻'],
 [1, '徒言树桃李此木岂无阴'],
 [2, '暮从碧山下山月随人归'],
 [2, '却顾所来径苍苍横翠微'],
 [2, '相携及田家童稚开荆扉'],
 [2, '绿竹入幽径青萝拂行衣'],
 [2, '欢言得所憩美酒聊共挥'],
 [2, '长歌吟松风曲尽河星稀'],
 [2, '我醉君复乐陶然共忘机'],
 [3, '花间一壶酒独酌无相亲'],
 [3, '举杯邀明月对影成三人'],
 [3, '月既不解饮影徒随我身'],
 [3, '暂伴月将影行乐须及春'],
 [3, '我歌月徘徊我舞影零乱'],
 [3, '醒时同交欢醉后各分散'],
 [3, '永结无情游相期邈云汉'],
 [4, '燕草如碧丝秦桑低绿枝'],
 [4, '当君怀归日是妾断肠时'],
 [5, '岱宗夫如何齐鲁青未了'],
 [5, '造化钟神秀阴阳割昏晓'],
 [5, '荡胸生层云决眦入归鸟'],
 [5, '会当凌绝顶一览众山小'],
 [6, '人生不相见动如参与商'],
 [6, '今夕复何夕共此灯烛光'],
 [6, '少壮能几时鬓发各已苍'],
 [6, '访旧半为鬼惊呼热中肠'],
 [6, '焉知二十载重上君子堂'],
 [6, '昔别君未婚儿女忽成行'],
 [6, '怡然敬父执问我来何方'],
 [6, '问答乃未已驱儿罗酒浆'],
 [6, '夜雨剪春韭新炊间黄粱'],
 [6, '主称会面难一举累十觞'],
 [6, '十觞亦不醉感子故意长'],
 [6, '明日隔山岳世事两茫茫'],
 [7, '绝代有佳人幽居在空谷'],
 [7, '自云良家女零落依草木'],
 [7, '关中昔丧乱兄弟遭杀戮'],
 [7, '官高何足论不得收骨肉'],
 [7, '世情恶衰歇万事随转烛'],
 [7, '夫婿轻薄儿新人美如玉'],
 [7, '合昏尚知时鸳鸯不独宿'],
 [7, '但见新人笑那闻旧人哭'],
 [7, '在山泉水清出山泉水浊'],
 [7, '侍婢卖珠回牵萝补茅屋'],


In [186]:
print(len(poem_string))
vocab_set = set(poem_string)
len(vocab_set)

5810


1385

## 1.2词频统计如下

In [187]:
print(len(poem_char_dic))
temp_a = sorted(poem_char_dic.items(),key=lambda x:x[1],reverse=True)
print(len(temp_a))
temp_a

1385
1385


[('山', 63),
 ('人', 59),
 ('不', 51),
 ('月', 44),
 ('风', 43),
 ('无', 41),
 ('归', 38),
 ('云', 38),
 ('来', 37),
 ('日', 37),
 ('何', 35),
 ('一', 32),
 ('天', 30),
 ('相', 29),
 ('江', 29),
 ('水', 29),
 ('自', 28),
 ('夜', 28),
 ('秋', 27),
 ('见', 27),
 ('生', 26),
 ('行', 26),
 ('长', 26),
 ('清', 26),
 ('此', 25),
 ('青', 25),
 ('君', 25),
 ('时', 25),
 ('中', 25),
 ('落', 25),
 ('白', 24),
 ('为', 23),
 ('客', 23),
 ('上', 23),
 ('下', 22),
 ('有', 21),
 ('心', 21),
 ('入', 21),
 ('我', 21),
 ('如', 21),
 ('空', 21),
 ('春', 20),
 ('南', 20),
 ('家', 20),
 ('花', 20),
 ('今', 20),
 ('道', 20),
 ('去', 20),
 ('欲', 20),
 ('寒', 19),
 ('别', 19),
 ('远', 19),
 ('孤', 19),
 ('明', 18),
 ('未', 18),
 ('雨', 18),
 ('故', 18),
 ('里', 18),
 ('处', 18),
 ('年', 18),
 ('暮', 17),
 ('独', 17),
 ('关', 17),
 ('高', 17),
 ('万', 17),
 ('门', 17),
 ('海', 17),
 ('意', 16),
 ('可', 16),
 ('深', 16),
 ('草', 16),
 ('阳', 16),
 ('几', 16),
 ('事', 16),
 ('西', 16),
 ('流', 16),
 ('知', 15),
 ('情', 15),
 ('与', 15),
 ('在', 15),
 ('还', 15),
 ('林', 14),
 ('闻', 14),
 ('重

## 1.3编码解码函数

In [188]:
vocab_to_int = {c: i for i, c in enumerate(vocab_set)}
int_to_vocab = dict(enumerate(vocab_set))
def encode_seq(seq):
    res=[]
    for i in seq:
        res.append(vocab_to_int[i])
    return np.array(res)
def decode_seq(arr):
    res=''
    for i in arr:
        res+=int_to_vocab[i]
    return res
a=encode_seq('长簟迎风早空城澹月华')
print(a)
b=decode_seq(a)
print(b)

[ 598  905  825 1034 1097  988 1064 1285  239 1056]
长簟迎风早空城澹月华


## 1.4 按5言 做Batch

In [189]:
# poem_seq_list
train_x_5=[]
train_y_5=[]
for [index,seq] in poem_seq_list:
    train_x_5.append(encode_seq(seq[:-1]))
    train_y_5.append(encode_seq(seq[1:]))
train_x_5=np.array(train_x_5)
train_y_5=np.array(train_y_5)
print(train_x_5.shape)
print(train_y_5.shape)
train_x_5=train_x_5[:580]
train_y_5=train_y_5[:580]
print(train_x_5.shape)
print(train_y_5.shape)


(581, 9)
(581, 9)
(580, 9)
(580, 9)


In [191]:

print(decode_seq(train_x_5[3]))
print(decode_seq(train_y_5[3]))


print(train_x_5[3])
print(train_y_5[3])

江南有丹桔经冬犹绿
南有丹桔经冬犹绿林
[ 707   48  256 1198  179 1354 1075 1052  519]
[  48  256 1198  179 1354 1075 1052  519 1293]


# 2 模型

In [192]:
vocab_size=len(vocab_set)
embedding_dim=400
rnn_units=600
batch_size=20

In [193]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model
model=build_model(vocab_size, embedding_dim, rnn_units, batch_size)

In [194]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [195]:
model.compile(optimizer=keras.optimizers.Adam(0.0001),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])

In [197]:
model(train_x_5[:20])

<tf.Tensor: id=95818, shape=(20, 9, 1385), dtype=float32, numpy=
array([[[-1.57238625e-03, -1.91559573e-03, -2.09968025e-03, ...,
         -1.60725717e-03, -3.09584837e-04, -9.75925010e-04],
        [-2.95461388e-03, -1.03603257e-03, -1.87915214e-03, ...,
         -7.83484895e-03,  8.02020263e-03, -1.26363914e-02],
        [-1.57135669e-02,  6.44631637e-03,  1.22094965e-02, ...,
         -1.67741161e-02,  8.02602712e-03, -4.22322238e-03],
        ...,
        [ 9.13194939e-03,  3.64519376e-03, -6.84000924e-03, ...,
         -2.03132373e-03,  3.71066108e-03, -7.37105496e-04],
        [ 9.02239792e-03,  1.56542864e-02, -1.09267030e-02, ...,
          7.42588006e-03, -5.17070200e-03, -1.29520253e-03],
        [ 6.34358590e-03,  1.48637984e-02, -1.29322987e-02, ...,
         -9.91830323e-03, -2.26657768e-03,  1.14975711e-02]],

       [[ 6.60388032e-03,  6.85473904e-04,  6.10314868e-03, ...,
         -8.11627600e-03, -2.23210268e-03, -9.25294310e-03],
        [ 1.00892847e-02, -5.71286306e

In [198]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (20, None, 400)           554000    
_________________________________________________________________
gru_10 (GRU)                 (20, None, 600)           1803600   
_________________________________________________________________
dense_10 (Dense)             (20, None, 1385)          832385    
Total params: 3,189,985
Trainable params: 3,189,985
Non-trainable params: 0
_________________________________________________________________


In [201]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints/gru_gushishengchengqi/'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [203]:
history = model.fit(train_x_5,train_y_5,batch_size=batch_size, epochs=80, callbacks=[checkpoint_callback])

Train on 580 samples
Epoch 1/100
580/580 [==============================] - 2s 4ms/sample - loss: 9.4497 - accuracy: 0.0153
Epoch 2/100
580/580 [==============================] - 2s 4ms/sample - loss: 9.2641 - accuracy: 0.0199
Epoch 3/100
580/580 [==============================] - 2s 4ms/sample - loss: 9.0841 - accuracy: 0.0218
Epoch 4/100
580/580 [==============================] - 2s 4ms/sample - loss: 8.8948 - accuracy: 0.0266
Epoch 5/100
580/580 [==============================] - 2s 4ms/sample - loss: 8.6994 - accuracy: 0.0314
Epoch 6/100
580/580 [==============================] - 2s 4ms/sample - loss: 8.4890 - accuracy: 0.0387
Epoch 7/100
580/580 [==============================] - 2s 4ms/sample - loss: 8.2425 - accuracy: 0.0500
Epoch 8/100
580/580 [==============================] - 2s 4ms/sample - loss: 7.9616 - accuracy: 0.0659
Epoch 9/100
580/580 [==============================] - 2s 4ms/sample - loss: 7.7688 - accuracy: 0.0764
Epoch 10/100
580/580 [==============================

KeyboardInterrupt: 

# 3  生成

In [204]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/gru_gushishengchengqi/ckpt_83'

In [205]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (1, None, 400)            554000    
_________________________________________________________________
gru_11 (GRU)                 (1, None, 600)            1803600   
_________________________________________________________________
dense_11 (Dense)             (1, None, 1385)           832385    
Total params: 3,189,985
Trainable params: 3,189,985
Non-trainable params: 0
_________________________________________________________________


In [213]:
def generate_text(model, start_string):
  # 评估步骤（用学习过的模型生成文本）

  # 要生成的字符个数
    num_generate = 4*10-len(start_string)

  # 将起始字符串转换为数字（向量化）
    input_eval = encode_seq(start_string)
    # 加一维
    input_eval = tf.expand_dims(input_eval, 0)
    print("input_eval",input_eval)

  # 空字符串用于存储结果
    text_generated = []

  # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
#         print(predictions.shape)
      # 删除批次的维度
        predictions = tf.squeeze(predictions, 0)
#         print(predictions.shape)
        
      # 用分类分布预测模型返回的字符
        predicted_id=tf.argmax(predictions.numpy().flatten()).numpy()
        print("max predicted_id",decode_seq([predicted_id]))
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        print("random predicted_id",decode_seq([predicted_id]))

      # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
        input_eval = tf.expand_dims([predicted_id], 0)
        print(input_eval.shape)

        text_generated.append(decode_seq([predicted_id]))

    poem = (start_string + ''.join(text_generated))
    print(poem)
    print(poem[:5],poem[5:10]) 
    print(poem[10:15],poem[15:20]) 
    print(poem[20:25],poem[25:30]) 
    print(poem[30:35],poem[35:40]) 


In [217]:
generate_text(model,"天寒")

input_eval tf.Tensor([[1172 1004]], shape=(1, 2), dtype=int32)
max predicted_id 地
random predicted_id 石
(1, 1)
max predicted_id 上
random predicted_id 都
(1, 1)
max predicted_id 代
random predicted_id 郭
(1, 1)
max predicted_id 代
random predicted_id 悴
(1, 1)
max predicted_id 代
random predicted_id 民
(1, 1)
max predicted_id 代
random predicted_id 捣
(1, 1)
max predicted_id 代
random predicted_id 豺
(1, 1)
max predicted_id 带
random predicted_id 殊
(1, 1)
max predicted_id 是
random predicted_id 蝶
(1, 1)
max predicted_id 是
random predicted_id 抵
(1, 1)
max predicted_id 代
random predicted_id 勉
(1, 1)
max predicted_id 空
random predicted_id 训
(1, 1)
max predicted_id 事
random predicted_id 悄
(1, 1)
max predicted_id 然
random predicted_id 望
(1, 1)
max predicted_id 秋
random predicted_id 难
(1, 1)
max predicted_id 代
random predicted_id 屠
(1, 1)
max predicted_id 代
random predicted_id 川
(1, 1)
max predicted_id 带
random predicted_id 田
(1, 1)
max predicted_id 带
random predicted_id 引
(1, 1)
max predicted_id 竿
random